In [1]:
import numpy
import numpy.random as rnd
import json
import pandas as pd
import itertools

In [2]:
%run logic.ipynb


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [3]:
class Participant:
    poll = None
    matrices = None
    epsilons = None
    budget = 0.
    cost = 0
    
    real_answer = {}
    
    def __init__(self, budget, poll):
        self.budget = budget # Initial privacy budget
        self.poll = poll
        
    def setMatrices(self, matrices):
        self.matrices = matrices
        
    def setEpsilons(self, epsilons):
        self.epsilons = epsilons
        self.cost = sum([self.epsilons[x] for x in self.epsilons.keys()])
        
    def createAnswers(self, ans_dist_dict): # ans_dist_dict = {qid:(Path, Fraction)}
        
        for question in ans_dist_dict.keys():
            answer_space, dist = zip(*ans_dist_dict[question].items())
            self.real_answer[question] = rnd.choice(answer_space, p=dist)
            
    def setAnswers(self, answers): # answers = {qid:path}
        for question in answers.keys():
            self.real_answer[question] = answers[question]
            
    def getAnswer(self):
        return self.real_answer
        
    def randomizedResponse(self):
        
        # Only reply if we can afford it
        responses = {}
        
        if (self.cost <= self.budget):
            self.budget = self.budget - self.cost
            
            for question in self.matrices.keys():
                rr_input = self.real_answer[question]
                transitions = [(x, a) for (x, a) in matrices[question].keys() if x==rr_input]
                probabilities = [matrices[question][t] for t in transitions]
                response_space = [out for (x, out) in transitions]
                
                response = rnd.choice(response_space, p=probabilities)
                responses[question] = response
            
            return responses
        
        # Budgets are public, so we can refuse to answer without leaking information
        return None
    
    def getTrueAnswers(self):
        return self.real_answer
    
    

# Answer according to dist
matrices = pollToMatrix(_example_json_poll)
epsilons = pollToEpsilon(_example_json_poll)

participant = Participant(120, _example_json_poll)
participant.setMatrices(matrices)
participant.setEpsilons(epsilons)

weights = pollToPathWeights(_example_json_poll)
ps = pollToPaths(_example_json_poll)

ans_dist_zip = {}
for question in ps.keys():
    paths = ps[question]
    weight = weights[question]
    zipped = []
    for path in paths:
         zipped.append((path, weight[pathToKey(path)]))
    ans_dist_zip[question] = zipped

participant.createAnswer(ans_dist_zip)
print participant.real_answer

participant.randomizedResponse()

# Given answer
matrices = pollToMatrix(_example_json_poll)
epsilons = pollToEpsilon(_example_json_poll)

participant = Participant(120, _example_json_poll)
participant.setMatrices(matrices)
participant.setEpsilons(epsilons)

paths = pollToPaths(_example_json_poll)

answers = {}
for question in paths:
    answers[question] = rnd.choice(paths[question])

participant.setAnswers(answers)
print participant.real_answer
participant.randomizedResponse()